In [ ]:
import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config
import time

from chanlun import kcharts
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange import get_exchange, Market
from chanlun.exchange.exchange_db import ExchangeDB
from chanlun import cl
from chanlun.xuangu import xuangu
import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config

"""
根据缠论数据，选择自己所需要的形态方法集合
"""

direction_types = {"long": ["down"], "short": ["up"]}
mmd_types = {
    "long": ["1buy", "2buy", "3buy", "l2buy", "l3buy"],
    "short": ["1sell", "2sell", "3sell", "l2sell", "l3sell"],
}


def get_opt_types(opt_type: list = []):
    if len(opt_type) == 0:
        opt_type = ["long"]
    opt_direction = list(
        itertools.chain.from_iterable([direction_types[x] for x in opt_type])
    )
    opt_mmd = list(itertools.chain.from_iterable([mmd_types[x] for x in opt_type]))
    return opt_direction, opt_mmd

def xingchenglidu(line: LINE) -> list:
    """
    计算线的行程力度（取绝对值）

    行程力度 = dy / dx
        dy = 终点与起点的绝对值差异
        dx = 线段之间的k线数量
    """
    if line.end.val == line.start.val:
        return 0

    dy = abs(line.end.val - line.start.val)
    dx = line.end.k.k_index - line.start.k.k_index
    return [dy, dx, dy/dx]

def interact():
    """执行后进入repl模式"""
    import code
    code.InteractiveConsole(locals=globals()).interact()
    
if __name__ == "__main__":
    from chanlun.exchange.exchange_tdx import ExchangeTDX
    from chanlun.exchange.exchange_db import ExchangeDB
    from chanlun.cl_utils import query_cl_chart_config, web_batch_get_cl_datas
    from chanlun.exchange.exchange import *

    market = "a"
    code = "SH.000300"
    freq = "5s"
    start_date="2025-02-19 09:30:00"
    end_date="2025-02-21 10:35:00"

    ex = ExchangeDB(market)
    cl_config = query_cl_chart_config(market, code)

    klines = ex.klines(code, freq, start_date, end_date)
    cds = web_batch_get_cl_datas(market, code, {freq: klines}, cl_config)

    cd = cds[0]
    
    xds = cd.get_xds()
    
    xd = xds[-1]
    
    bi_start = xd.start_line
    print(bi_start)
    dy_bi_start = abs(bi_start.end.val - bi_start.start.val)
    dx_bi_start = bi_start.end.k.k_index - bi_start.start.k.k_index
    xc_bi_start = dy_bi_start / dx_bi_start

    bi_end = xd.end_line
    print(bi_end)
    dy_bi_end = abs(bi_end.end.val - bi_end.start.val)
    dx_bi_end = bi_end.end.k.k_index - bi_end.start.k.k_index
    xc_bi_end = dy_bi_end / dx_bi_end

    print(dy_bi_start,dy_bi_end,dx_bi_start,dx_bi_end)
    print(xc_bi_end < xc_bi_start)

    
    if (dy_bi_end > dy_bi_start) and (xc_bi_end < xc_bi_start):
        print("段内笔背驰")
    




In [ ]:
renders = []
cl_datas = []

f = freq
klines = ex.klines(code, frequency=freq, start_date=start_date, end_date=end_date, args={"limit":9999})
klines = klines[:]
print(klines[:].head(1))
print(klines[:].tail(1))
print(f, '获取K线数据量：', len(klines))
_s = time.time()
cl_datas.append(cl.CL(code, f, cl_config).process_klines(klines[:]))
print('Run time: ', time.time() - _s)

for cd in cl_datas:
    title = '%s - 【%s】 周期数据图表' % (code, cd.frequency)
    renders.append(kcharts.render_charts(title, cd))

In [ ]:
renders[0]

In [ ]:
k1 = cd.get_src_klines()

In [ ]:
print(k1[1])

In [ ]:
zs2 = cd.get_xd_zss()[-2]
print(zs2.start.k.index)
print(k1[zs2.start.k.index])
print(zs2.end.k.index)

In [ ]:
last_zs = cd.get_last_xd_zs()
print(last_zs)

In [ ]:
xd_zss = cd.get_xd_zss()
for i in range(5):
    print(last_zs.lines[-i-1])

In [ ]:
for line in xd_zss[-1].lines:
    print(line)

In [ ]:
bi_zss = cd.get_bi_zss()
for i in range(5):
    print(bi_zss[-i])

In [ ]:
opts = None
cd_lv0 = cd
    # cd_lv1 = market_data.get_cl_data(code, market_data.frequencys[0])
if len(cd_lv0.get_xd_zss()) == 0:
    print(1111)

xd_lv0 = cd_lv0.get_xds()[-1]
xd_zss = cd_lv0.get_xd_zss()
xd_zs_lv0 = xd_zss[-1]
klines_lv0 = cd_lv0.get_klines()
kline_lv0 = cd_lv0.get_klines()[-1]
has_two_xd_zs = False
if (len(xd_zss) > 1):
    has_two_xd_zs = True
    xd_zs_second = xd_zss[-2]
bi_zs_lv0 = cd_lv0.get_bi_zss()[-1]

    # 2025-3-1: 必须是一段趋势初期的线段中枢，不能超过两个同方向中枢，否则只等反向趋势发起

if (
    (xd_zs_lv0.lines[-1].type == "up")
    ):
    if (xd_zs_lv0.type == "up" and xd_zs_lv0.lines[-1].bc_exists(["pz", "qs"], "|")):
        print(2222)
    if (has_two_xd_zs and xd_zs_second.type == "up"):
        for xd in xd_zs_lv0.lines:
            if xd.type == "up" and xd.bc_exists(["pz", "qs"], "|"):
                print(3333)
    opt_direction = "buy"
else:
    if (xd_zs_lv0.type == "down" and xd_zs_lv0.lines[-1].bc_exists(["pz", "qs"], "|")):
        print(4444)
    if (has_two_xd_zs and xd_zs_second.type == "down"):
        for xd in xd_zs_lv0.lines:
            if xd.type == "down" and xd.bc_exists(["pz", "qs"], "|"):
                print(5555)
    opt_direction = "sell"

    # 最新的线段要属于线段中枢内的线段 
    # 2025-2-21更新
    # 或新一根不回到中枢内的线段 
    # 限制线段内笔数量不超过3
print(xd_lv0)
#获取段中枢最后一个线段的后续反向一笔 
if xd_lv0.index == xd_zs_lv0.lines[-1].index :
    bi_lv0 = cd_lv0.get_bis()[-1] 
    print(bi_lv0)
    print(xd_lv0.end_line)
    if (bi_lv0.index > xd_lv0.end_line.index + 2
        # or xd_lv0.end_line.index - xd_lv0.start_line.index > 2
    ):
        print(6666)
# 2025-2-21 或者反向一段 ，获取段内最后一笔,线段内最多3笔
elif (xd_lv0.index == xd_zs_lv0.lines[-1].index + 1 
        and xd_lv0.end_line.index - xd_lv0.start_line.index <= 2 
    #   and xd_zs_lv0.lines[-1].end_line.index - xd_zs_lv0.lines[-1].start_line.index <= 2 
    ):
            
    bi_lv0 = xd_lv0.end_line
else:
    print(7777)
    
# 2025-2-27: 笔中枢起始要在线段中枢内
if bi_zs_lv0.start.index > xd_zs_lv0.end.index:
    print(8888)
# 2025-3-1: 三M点笔需在笔中枢后一笔之内
if bi_lv0.index > bi_zs_lv0.lines[-1].index + 2:
    print(9999)       
    # print(has_two_xd_zs,bi_lv0.type,bi_lv0.high,xd_zs_second.zg,bi_lv0.low,xd_zs_second.zd,bi_zs_lv0.lines[-1].low,bi_zs_lv0.lines[-1].high,)

# 2025-3-1：笔区间(当前收盘价和上一笔，也就是笔中枢离开笔)不要在倒数第二个线段区间内，过滤震荡行情
if (has_two_xd_zs and bi_lv0.type == "up" 
    and (
        (bi_lv0.high < xd_zs_second.zg and bi_lv0.low > xd_zs_second.zd) 
        or (bi_zs_lv0.lines[-1].low < xd_zs_second.zg and bi_zs_lv0.lines[-1].low > xd_zs_second.zd)
    )
):
    print(101010)
if (has_two_xd_zs and bi_lv0.type == "down" 
    and (
        (bi_lv0.high < xd_zs_second.zg and bi_lv0.low > xd_zs_second.zd)
        or (bi_zs_lv0.lines[-1].high < xd_zs_second.zg and bi_zs_lv0.lines[-1].high > xd_zs_second.zd)
    )
):
    print(111213)

# # 统计中枢震荡区间的用时（用lv1[日线] 的K线数量，来获取交易日时间）
# zd_days = len(
#     [_k for _k in cd_lv1.get_klines() if xd_zs_lv0.start.k.date <= _k.date]
# )
# if (self.zs_zd_days[0] <= zd_days <= self.zs_zd_days[1]) is False:
#     return opts

from chanlun.backtesting.base import *
class StrategyTupo(Strategy):
    def __init__(
        self, pre_zf_rate=None, zs_zd_days=None, ema_params=None, max_loss_rate=None
    ):
        super().__init__()
    def open():
        pass
    def close():
        pass

    def myfunc(self,cd_klines, use_close=True, bin_cnt=100, part=0.7):
        return self.get_frvp(cd_klines, use_close=True, bin_cnt=100, part=0.7)

stra = StrategyTupo()
# MA 判断
idx_ma120 = stra.idx_ma(cd_lv0, 120)
idx_ma250 = stra.idx_ma(cd_lv0, 250)
pos_df = {}
pos_df["__open_k_date"] = kline_lv0.date
# 操作方向是卖出，反向笔完成，且最高点低于中枢低点zd，形成线段中枢的次级别/本级别三卖，且最新k线是阴线，卖出
# 2025-2-21 更新，加入MA判断
print(kline_lv0)
print(opt_direction,
      bi_lv0.is_done(),
      bi_lv0.mmd_exists(["3sell"], "|"),
    (bi_lv0.high < xd_zs_lv0.zd),
    kline_lv0.c < kline_lv0.o,
    (idx_ma120[-1] < idx_ma250[-1]  and kline_lv0.c < idx_ma120[-1] ),
    idx_ma120[-1] < idx_ma120[-2],
    idx_ma120[-2] < idx_ma120[-3],
    abs(idx_ma120[-2] -idx_ma250[-2] ) < abs(idx_ma120[-1] -idx_ma250[-1])
)
if (
    opt_direction == "sell"
    and bi_lv0.is_done()
    and bi_lv0.mmd_exists(["3sell"], "|")
    and (bi_lv0.high < xd_zs_lv0.zd)
    and kline_lv0.c < kline_lv0.o
    and (idx_ma120[-1] < idx_ma250[-1]  and kline_lv0.c < idx_ma120[-1] )
    and idx_ma120[-1] < idx_ma120[-2] 
    and idx_ma120[-2] < idx_ma120[-3]
    and abs(idx_ma120[-2] -idx_ma250[-2] ) < abs(idx_ma120[-1] -idx_ma250[-1])
):
    # 2025-3-1  笔三卖点 要么在倒数第二个段中枢zg之上，还有下降空间，要么在倒数第二个段中枢zd之下，跌破前2个中枢，说明下降趋势强烈，反之，如果有倒数第二个段中枢，并且笔终点在倒数第二个中枢之间，则震荡可能性大，不参与
    if (has_two_xd_zs and (kline_lv0.c < xd_zs_second.zg) and (kline_lv0.c > xd_zs_second.zd)):
        print(121212)

    loss_price = xd_zs_lv0.zg
    open_pos_rate = stra.get_open_pos_rate(
        stra.max_loss_rate, kline_lv0.c, loss_price
    )
    # info["open_pos_rate"] = open_pos_rate  # 记录开仓占比
    opts.append(
        Operation(
            code,
            "buy",
            "3sell",
            loss_price,
            pos_df,
            f"价格突破 zd {xd_zs_lv0.zd}，做空卖出 {open_pos_rate}",
            pos_rate=open_pos_rate,
            open_uid=f"{kline_lv0.date}"
        )
    )        
print(kline_lv0.c,idx_ma120[-1],idx_ma250[-1],idx_ma120[-2],idx_ma120[-3])
print(opt_direction,
      bi_lv0.is_done(),
      bi_lv0.mmd_exists(["3buy"], "|"),
      (bi_lv0.low > xd_zs_lv0.zg),
      kline_lv0.c > kline_lv0.o,
        (kline_lv0.c  > idx_ma250[-1]  and kline_lv0.c > idx_ma120[-1] )
)
        # idx_ma120[-1] > idx_ma120[-2] ,
    #   idx_ma120[-2] > idx_ma120[-3],
    #   abs(idx_ma120[-2] -idx_ma250[-2] ) < abs(idx_ma120[-1] -idx_ma250[-1]))
# 操作方向是买入，反向笔完成，且最低点高于中枢高点zg，形成线段中枢的次级别三买，且最新k线是阳线，买入
if (
    opt_direction == "buy"
    and bi_lv0.is_done()
    and bi_lv0.mmd_exists(["3buy"], "|")
    and (bi_lv0.low > xd_zs_lv0.zg)
    and kline_lv0.c > kline_lv0.o
    and (kline_lv0.c  > idx_ma250[-1]  and kline_lv0.c > idx_ma120[-1] )
    # and idx_ma120[-1] > idx_ma120[-2] 
    # and idx_ma120[-2] > idx_ma120[-3]
    # and abs(idx_ma120[-2] -idx_ma250[-2] ) < abs(idx_ma120[-1] -idx_ma250[-1])
):
    # 2025-3-1  笔三买点 要么在倒数第二个段中枢zd之上，还有上升空间，要么在倒数第二个段中枢zg之上，突破前2个中枢，说明上升趋势强烈，反之，如果有倒数第二个段中枢，并且当前价格在倒数第二个中枢之间，则震荡可能性大，不参与
    if (has_two_xd_zs and (kline_lv0.c < xd_zs_second.zg) and (kline_lv0.c > xd_zs_second.zd)):
        print(131313)
    loss_price = xd_zs_lv0.zd
    open_pos_rate = stra.get_open_pos_rate(
        stra.max_loss_rate, kline_lv0.c, loss_price
    )
    # info["open_pos_rate"] = open_pos_rate  # 记录开仓占比
    opts.append(
        Operation(
            code,
            "buy",
            "3buy",
            loss_price,
            pos_df,
            f"价格突破 zg {xd_zs_lv0.zg}，做多买入 {open_pos_rate}",
            pos_rate=open_pos_rate,
            open_uid=f"{kline_lv0.date}"
        )
    )

print(151515)

In [ ]:
print(bc[0])

In [ ]:
bc = xd.get_bcs()
print(bc)

In [ ]:
lastbi = cd.get_bis()[-1]
print(lastbi)
print(bi_zss[-1].lines[0])
print(bi_zss[-1].start_line)

In [ ]:
bi = bi_zss[-1].lines[-1]
print(bi)

In [ ]:
bi.high > xd_zs.zd

In [ ]:
zs12 = bi_zss[13]

In [ ]:
print(zs12.lines[-1])

In [ ]:
print(xd.mmds[0])

In [ ]:
type(xd.start_line)

In [ ]:
zsds = cd.get_zsds()

In [ ]:
hasattr(zsds[0].start_line.start_line,'start_line')

In [ ]:
str(type(zsds[0].start_line.start_line.start_line))

In [ ]:
old_group = "测试选股-20241125-2216"
codes = zx.zx_stocks(zx_group)['code']

In [ ]:
str(cd.get_src_klines()[-2])

In [ ]:
str(cd.get_src_klines()[-1])

In [ ]:
zsds = cd.get_zsds()
zsd = zsds[-1]

In [ ]:
len(cd.get_idx()['macd']['dea'])


In [ ]:
macds_xd = cd.get_idx()['macd']['hist'][xd.start_line.start.k.k_index:xd.end_line.end.k.k_index + 1]

In [ ]:
xd.end_line.end.k.date
reference_date = pd.Timestamp('2024-07-01').tz_localize('UTC')
if reference_date <= xd.end_line.end.k.date:
    print(True)

In [ ]:
macd_xd = cd.get_idx()['macd']['dea'][bi_start.start.k.k_index:bi_start.end.k.k_index + 1]

In [ ]:
macds_xd = cd.get_idx()['macd']['hist'][xd.start_line.start.k.k_index:xd.end_line.end.k.k_index + 1]

In [ ]:
macds_xd

In [ ]:
macd_xd[-3] <= min(macd_xd[:])

In [ ]:
bizss = cd.get_bi_zss()

In [ ]:
(bizss[-1].zs_mmds())

In [ ]:
xd  = xds[-2]

In [ ]:
str(xd)

In [ ]:
bis = cd.get_bis()
for bi in bis:
    if bi.index == xd.start_line.index + 3:
        print(bi)
        print(bi.line_mmds())

In [ ]:
bis = cd.get_bis()
bis1 = bis[xd.start_line.index: xd.end_line.index + 1] 
for bi in bis1:
    print(bi)
    print(bi.line_mmds())

In [ ]:
type((bizss[-1].zs_mmds()[-2]))

In [ ]:
print(bizss[-2])

In [ ]:
bis[xd.start_line.index + 3].mmd_exists(["3sell"])


In [ ]:
bis[xd.start_line.index + 3].get_mmds()[0].name

In [ ]:
bi_start.end.k.date

In [ ]:
len(cd.get_klines())

In [ ]:
xingchenglidu(zsd)

In [ ]:
xingchenglidu(bi_start)

In [ ]:
xingchenglidu(bi_end)

In [ ]:
def xingchenglidu(line: LINE) -> list:
    """
    计算线的行程力度（取绝对值）

    行程力度 = dy / dx
        dy = 终点与起点的绝对值差异
        dx = 线段之间的k线数量
    """
    if line.end.val == line.start.val:
        return 0

    dy = abs(line.end.val - line.start.val)
    dx = line.end.k.k_index - line.start.k.k_index
    return [dy, dx, dy/dx]

def compare_xingcheng_ld(one_line: LINE, two_line: LINE):
    """
    比较两个线的力度，后者小于前者，返回 True
    :param one_line:
    :param two_line:
    :return:
    """
    dy1, dx1, ld1 = xingchenglidu(one_line)
    dy2, dx2, ld2 = xingchenglidu(two_line)
    if ( (dy2 > dy1) and  (ld2 < ld1)):
        return True
    else:
        return False

In [ ]:
compare_xingcheng_ld(zsd.start_line,zsd.end_line)

In [ ]:
zsd.start_line.end.k.date

In [ ]:
bi_start.jiaodu()

In [ ]:
xd.index

In [ ]:
xd.jiaodu()

In [ ]:
xd.start_line.high
# xd.start_line.low

In [ ]:
xd.end_line
xd.end_line.high
xd.end_line.low

In [ ]:
fx_start = xd.start
fx_end = xd.end

In [ ]:
fx_start.klines

In [ ]:
k_start = fx_start.k
k_end = fx_end.k

In [ ]:
k_start.k_index

In [ ]:
k_end.k_index

In [ ]:
diff = k_end.k_index - k_start.k_index


In [ ]:
diff